In [1]:
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,random_split
from torchvision import datasets, transforms

import pandas as pd
import numpy as np

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [3]:
data = pd.read_csv("dummyDataTopBottom0-10000.csv",header = None)
x=data.iloc[1:,1:5].astype('float32')
y=data.iloc[1:,-1].astype('float32')
x.iloc[:, 2] = x.iloc[:, 2] / 100
x.iloc[:, 3] = x.iloc[:, 3] / 100
y = y/max(y.values)

In [4]:
y

1        0.266667
2        0.400000
3        0.000000
4        0.066667
5        0.000000
           ...   
9996     0.000000
9997     0.433333
9998     0.533333
9999     0.000000
10000    0.966667
Name: 5, Length: 10000, dtype: float32

In [5]:
class WaterLevelDataset(Dataset):
    def __init__(self, x, y):
        x= torch.tensor(x,dtype=torch.float32)
        y = torch.tensor(y,dtype=torch.float32)
        self.X = x
        self.Y = y

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.Y[idx]]

In [6]:
dataset = WaterLevelDataset(x.values,y.values)

In [7]:
len(dataset.X)

10000

In [8]:
dataset.X

tensor([[0.0000, 1.0000, 0.9300, 0.3500],
        [0.0000, 0.0000, 0.6000, 0.6300],
        [0.0000, 1.0000, 0.2500, 0.3800],
        ...,
        [1.0000, 0.0000, 0.6700, 0.3200],
        [1.0000, 0.0000, 0.0900, 0.9800],
        [1.0000, 1.0000, 0.8000, 0.5400]])

In [9]:
dataset.Y

tensor([0.2667, 0.4000, 0.0000,  ..., 0.5333, 0.0000, 0.9667])

In [10]:
train, test = random_split(dataset, [int(len(dataset.X)*0.8),int(len(dataset.X)*0.2)])

In [11]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

## Model 1
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [12]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=self.layer2(X)
        X = self.activation(X)
        return X

# Instantiate the model
model = SimpleNN()



In [13]:
loss_fn = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)

In [14]:
for epoch in range(1000):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/1000], Loss: {loss.item()*100:0f}")



Epoch [1/250], Loss: 19.477706
Epoch [2/250], Loss: 13.862148
Epoch [3/250], Loss: 17.388648
Epoch [4/250], Loss: 12.003325
Epoch [5/250], Loss: 12.196782
Epoch [6/250], Loss: 12.286762
Epoch [7/250], Loss: 16.467601
Epoch [8/250], Loss: 12.261640
Epoch [9/250], Loss: 11.316510
Epoch [10/250], Loss: 13.320109
Epoch [11/250], Loss: 17.149219
Epoch [12/250], Loss: 14.453503
Epoch [13/250], Loss: 14.138693
Epoch [14/250], Loss: 15.120921
Epoch [15/250], Loss: 11.885679
Epoch [16/250], Loss: 11.011324
Epoch [17/250], Loss: 7.039240
Epoch [18/250], Loss: 11.376508
Epoch [19/250], Loss: 13.041957
Epoch [20/250], Loss: 12.630534
Epoch [21/250], Loss: 7.422644
Epoch [22/250], Loss: 7.256553
Epoch [23/250], Loss: 7.395019
Epoch [24/250], Loss: 8.070824
Epoch [25/250], Loss: 13.555643
Epoch [26/250], Loss: 10.801074
Epoch [27/250], Loss: 9.296586
Epoch [28/250], Loss: 10.221875
Epoch [29/250], Loss: 12.186247
Epoch [30/250], Loss: 7.641509
Epoch [31/250], Loss: 12.914790
Epoch [32/250], Loss: 12

In [15]:
model.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            outputs = model(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx])
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")


Predicted: 11.17, Actual: 2.00
Predicted: 3.62, Actual: 0.00
Predicted: 12.37, Actual: 16.00
Predicted: 2.35, Actual: 0.00
Predicted: 3.16, Actual: 0.00
Predicted: 7.46, Actual: 0.00
Predicted: 19.04, Actual: 13.00
Predicted: 3.75, Actual: 0.00
Predicted: 11.82, Actual: 4.00
Predicted: 11.81, Actual: 8.00
Predicted: 3.33, Actual: 0.00
Predicted: 16.44, Actual: 17.00
Predicted: 4.04, Actual: 0.00
Predicted: 1.86, Actual: 0.00
Predicted: 7.64, Actual: 0.00
Predicted: 4.80, Actual: 0.00
Average loss on the testing dataset: 2.9191
Predicted: 10.00, Actual: 18.00
Predicted: 4.00, Actual: 0.00
Predicted: 4.80, Actual: 26.00
Predicted: 1.44, Actual: 0.00
Predicted: 7.54, Actual: 14.00
Predicted: 1.34, Actual: 0.00
Predicted: 0.78, Actual: 0.00
Predicted: 0.47, Actual: 0.00
Predicted: 2.53, Actual: 0.00
Predicted: 2.41, Actual: 0.00
Predicted: 1.02, Actual: 0.00
Predicted: 3.74, Actual: 0.00
Predicted: 11.46, Actual: 23.00
Predicted: 6.46, Actual: 19.00
Predicted: 5.39, Actual: 5.00
Predicted:

In [16]:
torch.save(model, "model/model1TopBotomData10000-25.h5")

## Model 2
- Dataset used here is a dummy data with if top level more then 70 then predition should be 0

In [17]:
class Model2NN(nn.Module):
    def __init__(self):
        super(Model2NN, self).__init__()
        self.layer = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10,1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = torch.relu(self.layer(X))
        X=torch.relu(self.layer2(X))
        X=self.layer3(X)
        X = self.activation(X)
        return X

# Instantiate the model
model1 = Model2NN()



In [18]:
loss_fn = nn.MSELoss()

optimizer = optim.SGD(model1.parameters(), lr=0.001)

In [19]:
for epoch in range(1000):  # Number of epochs
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model1(inputs)
        loss = loss_fn(outputs.squeeze(), targets.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/10000], Loss: {loss.item()*100:0f}")



Epoch [1/250], Loss: 17.111504
Epoch [2/250], Loss: 15.124679
Epoch [3/250], Loss: 9.205971
Epoch [4/250], Loss: 13.209859
Epoch [5/250], Loss: 14.791776
Epoch [6/250], Loss: 13.766974
Epoch [7/250], Loss: 15.676764
Epoch [8/250], Loss: 10.609420
Epoch [9/250], Loss: 10.169192
Epoch [10/250], Loss: 14.231463
Epoch [11/250], Loss: 12.503614
Epoch [12/250], Loss: 12.468126
Epoch [13/250], Loss: 10.061267
Epoch [14/250], Loss: 11.009903
Epoch [15/250], Loss: 12.676449
Epoch [16/250], Loss: 7.211148
Epoch [17/250], Loss: 8.209468
Epoch [18/250], Loss: 7.626436
Epoch [19/250], Loss: 12.379772
Epoch [20/250], Loss: 8.663177
Epoch [21/250], Loss: 6.797534
Epoch [22/250], Loss: 12.201913
Epoch [23/250], Loss: 13.040495
Epoch [24/250], Loss: 16.003433
Epoch [25/250], Loss: 9.615223
Epoch [26/250], Loss: 12.931551
Epoch [27/250], Loss: 11.006681
Epoch [28/250], Loss: 13.974191
Epoch [29/250], Loss: 14.331345
Epoch [30/250], Loss: 8.091088
Epoch [31/250], Loss: 13.598169
Epoch [32/250], Loss: 15.

In [20]:
model1.eval()
with torch.no_grad():
    total_loss = 0
    num_samples = 0
    
    # Iterate through the testing data loader
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        for idx, test_inputs in enumerate(inputs):
            print(test_inputs)
            outputs = model1(test_inputs)
            # Optionally calculate the loss (if you want to evaluate performance)
            loss = loss_fn(outputs.squeeze(), targets[idx])
            
            # Accumulate the loss
            total_loss += loss.item()
            num_samples += 1
            
            # Print the predictions and the actual targets
            print(f"Predicted: {outputs.item()*30:.2f}, Actual: {targets[idx].item()*30:.2f}")
    
    # Calculate the average loss
        avg_loss = total_loss / num_samples
        print(f"Average loss on the testing dataset: {avg_loss*100:.4f}")

tensor([1.0000, 0.0000, 0.6500, 0.4200])
Predicted: 15.19, Actual: 2.00
tensor([1.0000, 1.0000, 0.0700, 0.2800])
Predicted: 1.83, Actual: 0.00
tensor([0.0000, 1.0000, 0.7200, 0.4000])
Predicted: 15.04, Actual: 16.00
tensor([0.0000, 0.0000, 0.5000, 0.8700])
Predicted: 1.37, Actual: 0.00
tensor([1.0000, 1.0000, 0.7100, 0.9700])
Predicted: 1.96, Actual: 0.00
tensor([1.0000, 1.0000, 0.0900, 0.0000])
Predicted: 3.08, Actual: 0.00
tensor([1.0000, 0.0000, 0.8300, 0.0200])
Predicted: 16.01, Actual: 13.00
tensor([1.0000, 1.0000, 0.2600, 0.4700])
Predicted: 4.11, Actual: 0.00
tensor([0.0000, 0.0000, 0.4700, 0.1700])
Predicted: 14.45, Actual: 4.00
tensor([0.0000, 1.0000, 0.4400, 0.1100])
Predicted: 14.52, Actual: 8.00
tensor([1.0000, 0.0000, 0.7100, 0.9600])
Predicted: 1.69, Actual: 0.00
tensor([0.0000, 0.0000, 0.7400, 0.1100])
Predicted: 15.33, Actual: 17.00
tensor([0.0000, 1.0000, 0.5900, 0.7600])
Predicted: 3.92, Actual: 0.00
tensor([1.0000, 0.0000, 0.3000, 0.7700])
Predicted: 1.39, Actual: 0.

In [21]:
torch.save(model1, "model/model2TopBotomData10000-25.h5")

In [23]:
test1 = torch.tensor([1,0,90/100,20/100])
print(test1)
outputs = model1(test1)
print(int(outputs.item()*30))

tensor([1.0000, 0.0000, 0.9000, 0.2000])
15
